# Task 1 - SQL testing

In [ ]:
import pandas as pd
import sqlite3

# Read Excel, header is on the 8th row (row index 7, since 0-based)
df = pd.read_excel("/content/drive/MyDrive/Delta/Data Engineer_Assessment_Data Set_Flight Leg.xlsx", header=7)
df['lastupdt'] = df['lastupdt'].astype(str)
# Create in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Insert dataframe into database
df.to_sql("my_table", conn, index=False, if_exists="replace")

# Query to check
cursor = conn.cursor()
cursor.execute("""
WITH RankedFlights AS (
    SELECT
        flightkey,
        flightnum,
        flight_dt,
        orig_arpt,
        dest_arpt,
        flightstatus,
        lastupdt,
        ROW_NUMBER() OVER (
            PARTITION BY flight_dt, flightnum, orig_arpt, dest_arpt
            ORDER BY lastupdt DESC, flightkey ASC  -- deterministic tie-breaker
        ) AS rn
    FROM my_table
)
SELECT
    flightkey,
    flightnum,
    flight_dt,
    orig_arpt,
    dest_arpt,
    flightstatus,
    lastupdt
FROM RankedFlights
WHERE rn = 1;


""")
print(cursor.fetchall())

conn.close()


# Task 2 Python function creation and testing

In [43]:
import pandas as pd

def get_latest_flight_status(file_path):
    """
    Reads an Excel file with flight data, auto-detects the header row,
    combines date and time into a proper datetime, drops carrier_code,
    and returns the most recent status for each flightkey.

    Parameters:
        file_path (str): Path to the Excel file.

    Returns:
        pd.DataFrame: Table with the most recent flight status for each flightkey.
    """
    # Read Excel without header
    df = pd.read_excel(file_path, header=None)

    # Detect header row by looking for 'flightkey'
    header_row = df.apply(lambda row: row.astype(str).str.lower().eq('flightkey').any(), axis=1).idxmax()

    if pd.isna(header_row):
        raise ValueError("Could not find the header row in the file.")

    # Assign column names from header row
    df.columns = df.iloc[header_row]

    # Keep only rows below the header
    df = df.iloc[header_row + 1:].reset_index(drop=True)

    # Drop carrier_code if present (case-insensitive)
    carrier_cols = [col for col in df.columns if str(col).strip().lower() in ["carrier code", "carrier_code"]]
    if carrier_cols:
        df = df.drop(columns=carrier_cols)

    # Ensure flight_dt is date and combine with lastupdt
    df['flight_dt'] = pd.to_datetime(df['flight_dt'], errors='coerce').dt.date
    df['lastupdt'] = pd.to_datetime(
        df['flight_dt'].astype(str) + " " + df['lastupdt'].astype(str),
        errors='coerce'
    )

    # Sort so most recent lastupdt is first per flightkey
    df = df.sort_values(by=['flightkey', 'lastupdt'], ascending=[True, False])

    # Deduplicate: keep only the most recent entry per flightkey
    latest_df = df.groupby('flightkey', as_index=False).first()

    return latest_df




In [44]:
# Example usage
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/Delta/Data Engineer_Assessment_Data Set_Flight Leg.xlsx"
    latest_status = get_latest_flight_status(file_path)
    print(latest_status)


7               flightkey   flight_dt flightnum orig_arpt dest_arpt  \
0       DL434661038ATLROC  2019-01-01      1038       ATL       ROC   
1       DL434661048ATLRSW  2019-01-01      1048       ATL       RSW   
2       DL434661050ATLDEN  2019-01-01      1050       ATL       DEN   
3       DL434661052ATLMKE  2019-01-01      1052       ATL       MKE   
4       DL434661053ATLPBI  2019-01-01      1053       ATL       PBI   
...                   ...         ...       ...       ...       ...   
244073   DL43830987ATLMSP  2019-12-31       987       ATL       MSP   
244074   DL43830991ATLDTW  2019-12-31       991       ATL       DTW   
244075   DL43830992ATLIAH  2019-12-31       992       ATL       IAH   
244076   DL43830993ATLMSP  2019-12-31       993       ATL       MSP   
244077   DL43830995ATLRDU  2019-12-31       995       ATL       RDU   

7      flightstatus            lastupdt  
0          Boarding 2019-01-01 09:28:00  
1          Boarding 2019-01-01 07:34:00  
2          Boarding 2

In [45]:
latest_status['flightkey'].nunique()

244078